### Domain Data

In [2]:
# Libraries
import os
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
import numpy as np
from pathlib import Path
import statsmodels.api as sm
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from fuzzywuzzy import process
from collections import Counter

In [3]:
os.getcwd()

'/Users/tasneemzulaiqa/Documents/GitHub/project-2-group-real-estate-industry-project-7-2025/Tas-workspace/notebooks'

In [4]:
domain_df = pd.read_csv("../../datasets/raw/cleaned/domain_cleaned.csv")
domain_df.head()

,sa2_code,sa2_name,suburb,postcode,weekly_rent,bond,address,lat,lon,bedrooms,...,swimming_pool,ensuite,dishwasher,garden,gym,pets_allowed,gas,intercom,security_system,washing_machine
0,213021344,Newport,SOUTH KINGSVILLE,3015,460.0,1994.0,3/53 Greene Street,-37.830982,144.87091,2,...,0,0,0,0,0,1,0,0,0,0
1,213021344,Newport,SOUTH KINGSVILLE,3015,400.0,1738.0,1/3 New Street,-37.826218,144.86755,2,...,0,0,0,0,0,1,0,0,0,1
2,213021343,Altona North,SOUTH KINGSVILLE,3015,795.0,3454.0,19/92 New Street,-37.831226,144.86632,3,...,0,1,1,0,0,1,1,0,0,1
3,213021344,Newport,SOUTH KINGSVILLE,3015,675.0,2933.0,3/14 Saltley Street,-37.827423,144.86768,3,...,0,0,1,0,0,0,0,0,0,0
4,213021344,Newport,SOUTH KINGSVILLE,3015,450.0,1955.0,4/2B Saltley Street,-37.826270,144.86790,2,...,0,0,0,0,0,0,0,0,0,0


In [8]:
domain_df['median_weekly_rent_sa2'] = domain_df.groupby('sa2_code')['weekly_rent'].transform('median')
domain_df

,sa2_code,sa2_name,suburb,postcode,weekly_rent,bond,address,lat,lon,bedrooms,...,ensuite,dishwasher,garden,gym,pets_allowed,gas,intercom,security_system,washing_machine,median_weekly_rent_sa2
0,213021344,Newport,SOUTH KINGSVILLE,3015,460.0,1994.0,3/53 Greene Street,-37.830982,144.87091,2,...,0,0,0,0,1,0,0,0,0,650.0
1,213021344,Newport,SOUTH KINGSVILLE,3015,400.0,1738.0,1/3 New Street,-37.826218,144.86755,2,...,0,0,0,0,1,0,0,0,1,650.0
2,213021343,Altona North,SOUTH KINGSVILLE,3015,795.0,3454.0,19/92 New Street,-37.831226,144.86632,3,...,1,1,0,0,1,1,0,0,1,670.0
3,213021344,Newport,SOUTH KINGSVILLE,3015,675.0,2933.0,3/14 Saltley Street,-37.827423,144.86768,3,...,0,1,0,0,0,0,0,0,0,650.0
4,213021344,Newport,SOUTH KINGSVILLE,3015,450.0,1955.0,4/2B Saltley Street,-37.826270,144.86790,2,...,0,0,0,0,0,0,0,0,0,650.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12613,208011169,Brighton (Vic.),BRIGHTON,3186,1575.0,6300.0,NaN,-37.912884,144.99155,2,...,1,1,0,0,1,0,0,0,1,1600.0
12614,208011169,Brighton (Vic.),BRIGHTON,3186,2625.0,10500.0,2/71 Roslyn Street,-37.922750,145.00224,4,...,0,0,0,0,0,0,0,0,0,1600.0
12615,208011169,Brighton (Vic.),BRIGHTON,3186,2200.0,13200.0,23 Bay Street,-37.903280,144.98697,5,...,0,0,0,0,0,0,0,0,0,1600.0
12616,208011169,Brighton (Vic.),BRIGHTON,3186,1390.0,8340.0,2/7B Wilson Street,-37.909650,144.99810,3,...,0,0,0,0,0,0,0,0,0,1600.0


In [12]:
sa2_median = domain_df.groupby('sa2_name')['weekly_rent'].median().reset_index()
sa2_median.rename(columns={'weekly_rent': 'median_weekly_rent'}, inplace=True)

# View
sa2_median.describe()

,median_weekly_rent
count,478.000000
mean,581.655596
std,138.807592
min,230.000000
25%,500.000000
50%,563.000000
75%,650.000000
max,1600.000000
